In [ ]:
import numpy as np
import matplotlib.cm as cm
import torch
import meshplot as mp
from model import SIREN
import open3d as o3d
import open3d.core as o3c
from PIL import Image
import json

In [ ]:
W0 = 30
LADO = 1024
SAMPLES = LADO ** 2
BORDES = [1, -1]
JOINT = 0
EJEPLANO = [0,2,1]
OFFSETPLANO = 0.0
MESH = 'armadillo_slice'
MODEL = 'model_best_hess2'

device = o3c.Device('CPU:0')
device_torch = torch.device(0)

with open(f'models/{MESH}/{MESH}.json') as jsonFile:
	skel = json.load(jsonFile)
	features = len(skel['joints'][0]['mean'])

	mesh = o3d.t.geometry.TriangleMesh(device)
	mesh.vertex["positions"] = o3c.Tensor(np.array(skel['joints'][JOINT]['vertices']), dtype=o3c.float32)
	mesh.triangle["indices"] = o3c.Tensor(np.array(skel['joints'][JOINT]['triangles']), dtype=o3c.int32)

	model = SIREN(
			n_in_features= features + 3,
			n_out_features=1,
			hidden_layer_config=[256,256,256,256],
			w0=W0,
			ww=None
	)
	model.load_state_dict( torch.load(f'models/{MESH}/{MODEL}.pth', map_location=device_torch))
	model.to(device_torch)

	i_1, i_2 = np.meshgrid( np.linspace(BORDES[0], BORDES[1], LADO), np.linspace(BORDES[0], BORDES[1], LADO) )
	samples = np.concatenate(
			np.concatenate( np.array([np.expand_dims(i_1, 2), 
								np.expand_dims(i_2, 2), 
								np.expand_dims(np.ones_like(i_1) * OFFSETPLANO, 2)])[EJEPLANO]
						, axis=2 ),
			axis=0)

	code = np.tile( skel['joints'][JOINT]['mean'], (SAMPLES, 1))
	inputs = np.hstack([code, samples])

	distancias_pred = np.zeros(SAMPLES)
	gradient_norm = np.zeros(SAMPLES)
	gradient_colormap = np.zeros((SAMPLES, 3))
	max_batch = 64 ** 3

	head = 0
	while head < SAMPLES:
		# print(head)
		inputs_subset = torch.from_numpy(inputs[head:min(head + max_batch, SAMPLES), :]).float().to(device_torch)
		x, y =  model(inputs_subset).values()
		df_pred = y #torch.clamp( y , max=self.threshold )
		df_pred.sum().backward()
		
		indexes = torch.tensor( [features, features + 1, features + 2] ).to(device_torch)
		gradient = torch.index_select( x.grad, 1, indexes)
		norm = torch.linalg.norm(gradient, dim=-1)

		stacked_norm = torch.hstack([ norm.unsqueeze(1), norm.unsqueeze(1), norm.unsqueeze(1)])
		normalized_grad = torch.where(
			stacked_norm < 0.025,
			torch.ones_like(stacked_norm) * -1,
			torch.div( gradient, stacked_norm) )


		distancias_pred[head:min(head + max_batch, SAMPLES)] = y.squeeze().detach().cpu()
		gradient_norm[head:min(head + max_batch, SAMPLES)] = norm.detach().cpu().numpy()
		gradient_colormap[head:min(head + max_batch, SAMPLES)] = ((normalized_grad + torch.ones_like(normalized_grad) ) / 2).detach().cpu().numpy()
		
		head += max_batch

	scene = o3d.t.geometry.RaycastingScene()
	scene.add_triangles(mesh)
	distancias_gt = scene.compute_distance( o3c.Tensor(samples, dtype=o3c.float32) ).numpy() ** 2


In [ ]:
print( f'Rango pred: [{np.min(distancias_pred)}, {np.max(distancias_pred)}]')
print( f'Rango gt: [{np.min(distancias_gt)}, {np.max(distancias_gt)}]')

In [ ]:
vertices_plano = np.array([
    np.array([BORDES[1], BORDES[1], OFFSETPLANO])[EJEPLANO],
    np.array([BORDES[1], BORDES[0], OFFSETPLANO])[EJEPLANO],
    np.array([BORDES[0], BORDES[0], OFFSETPLANO])[EJEPLANO],
    np.array([BORDES[0], BORDES[1], OFFSETPLANO])[EJEPLANO]
])
triangulos_plano = np.array([[0,1,2],[2,3,0]])
p = mp.plot(mesh.vertex['positions'].numpy(), mesh.triangle['indices'].numpy(), return_plot=True)
p.add_mesh( vertices_plano,triangulos_plano, c=np.array([1,0,0]))

In [ ]:

mesh = o3d.geometry.TriangleMesh( o3d.utility.Vector3dVector(vertices_plano), o3d.utility.Vector3iVector(triangulos_plano))
o3d.io.write_triangle_mesh('plano.obj', mesh)

In [ ]:
def imagen_dist( path, distancias, niveles, eps=0.0005, negs=False, color_map='br'):
    def lines( distancias, niveles, eps ):
        res = np.ones_like(distancias, dtype=np.bool8)
        
        for v in niveles:
            res = np.logical_and( np.logical_not( np.logical_and( distancias > v - eps, distancias < v + eps)), res)

        return res


    mask = lines(distancias, niveles, eps)

    if color_map == 'br':
        cmap = cm.get_cmap('binary')
        colores = np.expand_dims(cmap( distancias[mask] )[:, 0], axis=1)
        colores = np.hstack( [np.ones_like(colores) - colores, np.zeros_like(colores), colores])
    else:
        cmap = cm.get_cmap(color_map)
        colores = cmap( distancias[mask] )[:, :3]

    imagen = np.ones((SAMPLES,3))
    imagen[mask] = colores

    if negs:
        imagen[distancias < 0] = np.tile( np.eye( 1, 3, k=1) * 255, (sum(distancias < 0),1)).astype(np.uint32)

    im = Image.fromarray((imagen.reshape(np.sqrt(SAMPLES).astype(np.uint32), np.sqrt(SAMPLES).astype(np.uint32), 3) * 255).astype(np.uint8))
    im.save(path, 'PNG')

imagen_dist( 'field_pred.png',distancias_pred / np.max(distancias_pred), np.linspace(0,1,10), negs=True, color_map='turbo', eps=0.001)
#imagen_dist( 'field_gt.png',distancias_gt / np.max(distancias_gt), np.linspace(0,1,10), negs=True, color_map='turbo', eps=0.001)
imagen_dist( 'grad_norm.png',gradient_norm / np.max(gradient_norm), np.linspace(0,1,10), color_map='turbo', eps=0.001 )

im = Image.fromarray((gradient_colormap.reshape(np.sqrt(SAMPLES).astype(np.uint32), np.sqrt(SAMPLES).astype(np.uint32), 3) * 255).astype(np.uint8))
im.save('gradient_map.png', 'PNG')

In [ ]:
np.min(gradient_norm), np.max(gradient_norm)